In [ ]:
import os,json

result = {}
root_path = r"/root/autodl-tmp/prompt-matcher-reduce-uncertainty/Valentine-datasets"
o = ["assays", "miller2", "prospect"]
m = ["Joinable", "Semantically-Joinable", "Unionable","View-Unionable"]
for i in o:
    tmp_list = []
    dir_path = os.path.join(root_path, i)
    for j in m:
        final_dir_path = os.path.join(dir_path, j)
        for _,files_path,_ in os.walk(final_dir_path):
            for files_path_item in files_path:
                source_json = os.path.join(os.path.join(final_dir_path, files_path_item), files_path_item) + "_source.json"
                target_json = os.path.join(os.path.join(final_dir_path, files_path_item), files_path_item )+ "_target.json"
                with open(source_json, "r") as f:
                    content = json.load(f)
                    tmp_list.extend(set(list(content.keys())))
                with open(target_json, "r") as f:
                    content=json.load(f)
                    tmp_list.extend(set(list(content.keys())))
    result[i] = set(tmp_list)

In [ ]:
len(result["assays"]), len(result["miller2"]),len(result["prospect"])

In [ ]:
import json
import os
import openai
openai.api_base = "https://openkey.cloud/v1"
openai.api_key = "sk-wdPR1ALmXgg4XEjhE9136729A60447F4851133C3E7Da4590"
def query_chatgpt(message_param):
    sentence = openai.ChatCompletion.create(
                                    model="gpt-4-0613",
                                    messages=message_param,
                                    # 流式输出
                                    temperature=0.1,
                                    stream = False)
    
    return sentence["choices"][0]["message"]["content"]

def len_data(path_dir):
    data = []
    for i in ["train.json", "test.json", "valid.json"]:
        with open(os.path.join(path_dir,i), "r", encoding="utf-8") as f:
            content = json.load(f)
        print(f"{i} 长度:", len(content))
        data.append(content)
    return data

In [ ]:
path = r"./data/fabricated_dataset"
train_data, test_data, dev_data = len_data(path)

In [ ]:
attribute_l = []
for i in test_data:
    attribute_l.append(i[0].split("[VAL]")[0].replace("[ATT]", "").strip())
    attribute_l.append(i[1].split("[VAL]")[0].replace("[ATT]", "").strip())


In [ ]:
dic_name = {}
for att in attribute_l:
    exist_bool = False
    for key_n in result.keys():
        if att in result[key_n]:
            exist_bool = True
            dic_name[att.lower()] = key_n
    if not exist_bool:
        print(att)

In [ ]:
# Help determine if the attributes should match or not in the schema match task. 
# Please resume the attribute abbreviation first, if it exists. The attributes of two schemas to store {schema_name} information are: 
# {attribute_name}  and  {attribute_name2} answer with [yes or no ] and give a confidence value in (0.0, 1.0). Demonstrate the reasons, 
#################### 80% acc
# Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
# Tips:
# (1) Abbreviations are generated by extracting letters from the full name. Please Resume the attribute abbreviation first, if it exists. 
# (2) These two schemas are used to store {schema_name} information.
# (3) if the value instances belong the same type of data and are quite similar with each other, this will help to identify the match.
# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no] and give a probability of answer as ground truth.
## 88 % acc
# Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
# Tips:
# (1) Some letters are extracted from the full names and merged into a abbreviation word. Schema name sometimes are also added as the prefix of abbreviation.
# (2) Please Resume the attribute abbreviation first, if it exists. 
# (2) These two schemas are used to store {schema_name} information.
# (3) match may be likely correct, if the second value instances are also suitable for the first attribute name. 
# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no] and give a probability of answer as ground truth.
# 90% acc

# Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
# Tips:
# (1) Some letters are extracted from the full names and merged into an abbreviation word.
# (2) Schema information sometimes is also added as the prefix of abbreviation.
# (3) Please consider the abbreviation case.
# (4) These two schemas are used to store {schema_name} information.
# (5) values exchange verification: match would be likely correct, if the second value instances are also suitable for the first attribute name. 
# (6) A match is correct as long as one of the name and values exchange verification is correct
# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no] and give a probability of answer as ground truth.
# 94.5%  0.9028967768257854
# Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
# Tips:
# (1) The names of two schemas are both {schema_name}.
# (2) if there is an abbreviation of attribute name. Please check the abbreviation. if it has schema name as its prefix, then Ignore the prefixes and compare the suffixes.
# Then if the suffixes are a set of letters extracted from another attribute name, the abbreviation and attribute name may probably match.
# (3) Values Exchange Verification: match would be likely correct, if the second value instances are also suitable for the first attribute name. 
# (6) A match is correct as long as one of the name and Values Exchange Verification is correct

# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no].

In [ ]:
# Determine whether the two attributes can be took as the same attribute in schema match. 
# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no].
# 0.71 100、
# Determine whether the two attributes can be took as the same attribute in schema match. 
# (1) Values Exchange Verification: match would be likely correct, if the second value instances are also suitable for the first attribute name. 
# (2) A match is correct as long as one of the name and Values Exchange Verification is correct
# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no].
# 0.75 100
# Determine whether the two attributes can be took as the same attribute in schema match.Remember some tips.
# Tips:
# (1) The names of two schemas are both {schema_name}.
# (2) if there is an abbreviation of attribute name.If abbreviation has schema name as its prefix, ignore the prefix.
#  Then if the suffix is a set of letters extracted from another attribute name, the abbreviation and attribute name may probably match.
# (3) Values Exchange Verification: match would be likely correct, if the second value instances are also suitable for the first attribute name. 
# (4) A match is correct as long as one of the name and Values Exchange Verification is correct
# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no].
# 0.86 100

# Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
# Tips:
# (1) Some letters are extracted from the full names and merged into an abbreviation word.
# (2) Schema information sometimes is also added as the prefix of abbreviation.
# (3) Please consider the abbreviation case.
# (4) These two schemas are used to store {schema_name} information.
# (5) values exchange verification: match would be likely correct, if the second value instances are also suitable for the first attribute name. 
# (6) A match is correct as long as one of the name and values exchange verification is correct
# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no] and give a probability of answer as ground truth.
# 0.91 100

# 300 (4)(5)(6) 0.86
# 300 (1)(2)(3)(5)(6) 0.92
# 300 (1)(2)(3)(4)(5)(6) 0.91
# 300 (1)(2) 0.806
# 300 (1)(2)(3) 0.8533
# 300 (1)(2)(3)(4) 0.82
# 300 (1)(2)(3)(5) 0.913 / 0.918 for 2451
# 300 (1)(2)(3)(5)(6) no probability 0.907
# 0.918
# Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
# Tips:
# (1) These two schemas are used to store {schema_name} information
# (2) Some letters are extracted from the full names and merged into an abbreviation word.
# (3) Schema information sometimes is also added as the prefix of abbreviation.
# (4) values exchange verification: match would be likely correct, if the second value instances are also suitable for the first attribute name.
# Input:
# First Attribute Name: {attribute_name} 
# its Value instances: {values} 

# Second Attribute Name: {attribute_name2} 
# Its Value instances: {values2}. \n
# Please answer with [yes or no].

# Semantic-match Prompt:

# Determine whether the two attributes match with each other in schema match.
# Input:
# First Attribute Name: {attribute_name}
# Second Attribute Name: {attribute_name2}
# Please answer with [yes or no].

In [41]:
def prompt_add(list_item, fabricate=True):
    input_str = list_item[0].split("[VAL]")
    attribute_name = input_str[0].replace("[ATT]","")
    if fabricate:
        schema_name = dic_name[attribute_name.strip().lower()]
    attribute_name=f"{attribute_name.strip().lower()}"
    values = [i.strip() for i in input_str[17:20]]
    
    input_str2 = list_item[1].split("[VAL]")
    attribute_name2 = input_str2[0].replace("[ATT]", "")
    attribute_name2=f"{attribute_name2.strip().lower()}"
    values2 = [i.strip() for i in input_str2[16:19]]
    return f''' 
Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
Tips:
(1) These two schemas are used to store {schema_name} information
(2) Some letters are extracted from the full names and merged into an abbreviation word.
(3) Schema information sometimes is also added as the prefix of abbreviation.
(4) values exchange verification: match would be likely correct, if the second value instances are also suitable for the first attribute name.
Input:
First Attribute Name: {attribute_name} 
its Value instances: {values} 

Second Attribute Name: {attribute_name2} 
Its Value instances: {values2}. \n
Please answer with [yes or no].
''', list_item[2]

In [ ]:
# 1 6 8 17 28 54 55 80

In [42]:
import random
from tqdm import tqdm
test_true_n = 0
test_data = test_data[:20]
for item_data in tqdm(test_data):
    prompt, label = prompt_add(item_data)
    mes = [{"role":"user", "content":prompt}]
    response_str = query_chatgpt(mes)
    if "yes" in response_str or "Yes" in response_str:
        if label==1.0:
            test_true_n+=1
        else:
            print(prompt)
            print("################")
            print(response_str)
            print("####################")
            print(label)
    elif "no" in response_str or "No" in response_str:
        if label==0:
            test_true_n+=1
        else:
            print(prompt)
            print("################")
            print(response_str)
            print("################")
            print(label)

  5%|▌         | 1/20 [00:12<04:06, 12.98s/it]

 
Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
Tips:
(1) These two schemas are used to store prospect information
(2) Some letters are extracted from the full names and merged into an abbreviation word.
(3) Schema information sometimes is also added as the prefix of abbreviation.
(4) values exchange verification: match would be likely correct, if the second value instances are also suitable for the first attribute name.
Input:
First Attribute Name: agencyid 
its Value instances: ['CHA4155', 'Jug6661', 'Kxz6241'] 

Second Attribute Name: gncd 
Its Value instances: ['Gvq11342', 'CXN12444', 'imw9156']. 

Please answer with [yes or no].

################
No
################
1


 30%|███       | 6/20 [00:20<00:27,  1.99s/it]

 
Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
Tips:
(1) These two schemas are used to store prospect information
(2) Some letters are extracted from the full names and merged into an abbreviation word.
(3) Schema information sometimes is also added as the prefix of abbreviation.
(4) values exchange verification: match would be likely correct, if the second value instances are also suitable for the first attribute name.
Input:
First Attribute Name: addressline1 
its Value instances: ['14148 saxon park', '3242 whistler park', '12813 bolton court'] 

Second Attribute Name: ddrssln1 
Its Value instances: ['BTFHGQCEZUYMNYVWNCV', 'JWFGKKNNWOYR', 'HCXXLCIPIHF']. 

Please answer with [yes or no].

################
no
################
1


 35%|███▌      | 7/20 [00:20<00:20,  1.61s/it]

 
Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
Tips:
(1) These two schemas are used to store prospect information
(2) Some letters are extracted from the full names and merged into an abbreviation word.
(3) Schema information sometimes is also added as the prefix of abbreviation.
(4) values exchange verification: match would be likely correct, if the second value instances are also suitable for the first attribute name.
Input:
First Attribute Name: addressline2 
its Value instances: ['Apt. 426', 'apt. 803', 'dxflsbamo'] 

Second Attribute Name: ddrssln2 
Its Value instances: ['Glxbwflprgr', 'Sajcdy', 'rqnjaezkocsjvbvhhfr']. 

Please answer with [yes or no].

################
No
################
1


 85%|████████▌ | 17/20 [00:41<00:03,  1.31s/it]

 
Determine the two attributes can be took as the same attribute in schema match. Remember some tips.
Tips:
(1) These two schemas are used to store prospect information
(2) Some letters are extracted from the full names and merged into an abbreviation word.
(3) Schema information sometimes is also added as the prefix of abbreviation.
(4) values exchange verification: match would be likely correct, if the second value instances are also suitable for the first attribute name.
Input:
First Attribute Name: age 
its Value instances: ['14.0', '4.0', '84.0'] 

Second Attribute Name: g 
Its Value instances: ['20.0', '23.0', '91.0']. 

Please answer with [yes or no].

################
no
################
1


100%|██████████| 20/20 [00:55<00:00,  2.78s/it]


In [ ]:
float((test_true_n)/len(test_data))

In [ ]:
len(test_data)

In [ ]:
test_true_n/1257